## Prepair

In [1]:
## config.yamlの読み込み
import yaml
with open("../scripts/config.yaml", "r", encoding='utf-8') as file:
    config = yaml.safe_load(file)

In [2]:
import gc
import lightgbm as lgb
from sklearn.ensemble import VotingRegressor
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import random
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, cohen_kappa_score
from lightgbm import log_evaluation, early_stopping
import polars as pl
from pathlib import Path
import sys
import torch

In [3]:
# 自作関数の読み込み
repo_dir = Path().resolve().parents[0]
root_dir = Path().resolve().parents[1]
s3_dir = root_dir / "s3storage/01_public/auto_essay_scorer_lab2/data/"
sys.path.append(str(repo_dir / "scripts/"))
from utils.path import PathManager
from utils.data import *
from utils.model import *

## パスの設定
mode = config["model_name"]
path_to = PathManager(s3_dir, mode)

2024-05-18 15:43:12.289387: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 15:43:12.289508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 15:43:12.428849: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-18 15:43:15.590307: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# a_valueとb_valueを確認

In [4]:
## 学習データ読み込み
load_path = path_to.train_all_mart_dir
train_data = pl.read_csv(load_path)

In [5]:
# a_value : 2.998
mean_score = train_data['score'].mean()
print(f"Average of column 'score': {mean_score}")

# b_value : 1.092
variance_score = train_data['score'].var()
print(f"Variance of column 'score': {variance_score}")

Average of column 'score': 2.948402380539666
Variance of column 'score': 1.0918134361390224


# scripts/01_prepare.py 実行後のデータ確認

In [30]:
## データの読み込み
fold_num = "0"

train_path: Path = path_to.add_meta_mart_dir / f'fold_{fold_num}/train_fold_add_meta.csv'
train_data = pl.read_csv(train_path)
valid_path: Path = path_to.add_meta_mart_dir / f'fold_{fold_num}/valid_fold_add_meta.csv'
valid_data = pl.read_csv(valid_path)

In [31]:
train_data.head(1)

,essay_id,paragraph_>0_cnt,paragraph_>50_cnt,paragraph_>75_cnt,paragraph_>100_cnt,paragraph_>125_cnt,paragraph_>150_cnt,paragraph_>175_cnt,paragraph_>200_cnt,paragraph_>250_cnt,paragraph_>300_cnt,paragraph_>350_cnt,paragraph_>400_cnt,paragraph_>500_cnt,paragraph_>600_cnt,paragraph_>700_cnt,paragraph_<25_cnt,paragraph_<49_cnt,paragraph_error_num_max,paragraph_len_max,paragraph_sentence_cnt_max,paragraph_word_cnt_max,paragraph_error_num_mean,paragraph_len_mean,paragraph_sentence_cnt_mean,paragraph_word_cnt_mean,paragraph_error_num_min,paragraph_len_min,paragraph_sentence_cnt_min,paragraph_word_cnt_min,paragraph_error_num_sum,paragraph_len_sum,paragraph_sentence_cnt_sum,paragraph_word_cnt_sum,paragraph_error_num_first,paragraph_len_first,…,tfid_cnt_2136,tfid_cnt_2137,tfid_cnt_2138,tfid_cnt_2139,tfid_cnt_2140,tfid_cnt_2141,tfid_cnt_2142,tfid_cnt_2143,tfid_cnt_2144,tfid_cnt_2145,tfid_cnt_2146,tfid_cnt_2147,tfid_cnt_2148,tfid_cnt_2149,tfid_cnt_2150,tfid_cnt_2151,tfid_cnt_2152,tfid_cnt_2153,tfid_cnt_2154,tfid_cnt_2155,tfid_cnt_2156,tfid_cnt_2157,tfid_cnt_2158,tfid_cnt_2159,tfid_cnt_2160,tfid_cnt_2161,tfid_cnt_2162,tfid_cnt_2163,tfid_cnt_2164,tfid_cnt_2165,tfid_cnt_2166,tfid_cnt_2167,tfid_cnt_2168,tfid_cnt_2169,tfid_cnt_2170,tfid_cnt_2171,tfid_cnt_2172
i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
0,"""0040e27""",5,5,5,5,5,5,5,4,4,2,1,1,0,0,0,0,0,2,424,7,80,1.4,300.0,5.6,55.2,1,179,5,31,7,1500,28,276,1,179,…,0,2,0,0,0,2,0,0,0,0,0,0,1,0,1,1,0,0,2,0,1,1,1,12,0,12,0,7,7,0,0,0,0,0,0,0,0


In [32]:
valid_data.head(1)

,essay_id,paragraph_>0_cnt,paragraph_>50_cnt,paragraph_>75_cnt,paragraph_>100_cnt,paragraph_>125_cnt,paragraph_>150_cnt,paragraph_>175_cnt,paragraph_>200_cnt,paragraph_>250_cnt,paragraph_>300_cnt,paragraph_>350_cnt,paragraph_>400_cnt,paragraph_>500_cnt,paragraph_>600_cnt,paragraph_>700_cnt,paragraph_<25_cnt,paragraph_<49_cnt,paragraph_error_num_max,paragraph_len_max,paragraph_sentence_cnt_max,paragraph_word_cnt_max,paragraph_error_num_mean,paragraph_len_mean,paragraph_sentence_cnt_mean,paragraph_word_cnt_mean,paragraph_error_num_min,paragraph_len_min,paragraph_sentence_cnt_min,paragraph_word_cnt_min,paragraph_error_num_sum,paragraph_len_sum,paragraph_sentence_cnt_sum,paragraph_word_cnt_sum,paragraph_error_num_first,paragraph_len_first,…,tfid_cnt_2136,tfid_cnt_2137,tfid_cnt_2138,tfid_cnt_2139,tfid_cnt_2140,tfid_cnt_2141,tfid_cnt_2142,tfid_cnt_2143,tfid_cnt_2144,tfid_cnt_2145,tfid_cnt_2146,tfid_cnt_2147,tfid_cnt_2148,tfid_cnt_2149,tfid_cnt_2150,tfid_cnt_2151,tfid_cnt_2152,tfid_cnt_2153,tfid_cnt_2154,tfid_cnt_2155,tfid_cnt_2156,tfid_cnt_2157,tfid_cnt_2158,tfid_cnt_2159,tfid_cnt_2160,tfid_cnt_2161,tfid_cnt_2162,tfid_cnt_2163,tfid_cnt_2164,tfid_cnt_2165,tfid_cnt_2166,tfid_cnt_2167,tfid_cnt_2168,tfid_cnt_2169,tfid_cnt_2170,tfid_cnt_2171,tfid_cnt_2172
i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
0,"""0f2a7d9""",4,4,4,4,4,4,4,3,3,3,2,2,1,0,0,0,0,5,558,4,101,2.5,370.0,3.0,68.25,1,188,2,35,10,1480,12,273,2,405,…,0,0,1,0,5,3,0,0,1,0,2,2,1,0,1,1,1,0,0,0,0,2,2,3,0,3,0,1,1,0,0,0,0,0,0,0,0


In [37]:
len(train_data)

93

In [ ]:
valid_data

In [36]:
len(train_data.columns)

24143

In [34]:
missing_columns = [col for col in ['Unnamed: 0'] if col not in train_data.columns]

In [35]:
missing_columns

['Unnamed: 0']